### Open the files
Select the 0th index counts file to be read in

In [1]:
from t3r_read import t3r
from tdms_read import tdms
import os;
from tkinter import filedialog
from zoom import zoom_factory;
import numpy as np;
from matplotlib.widgets import Slider,SpanSelector, Button
import matplotlib.pyplot as plt


%load_ext autoreload
%autoreload 2

file_path = filedialog.askopenfilename();
filename = os.path.abspath(file_path);

In [89]:
from plotter import Plotter
from dataclasses import dataclass

from utils import make_extension, AbstractFile


use_inputsgn_files = True
file_interval = (0,5)  # (n,m) open file with indices starting at n up to m-1
seconds_in_file = 10.1  # use this value if it is desired to override standard calculation of file length

abstractfile = AbstractFile(filename, make_extension, '00')
    
plotter = Plotter(abstractfile, file_interval=file_interval, bin=30e-3)
seconds_in_file = plotter.seconds_in_file

In [93]:
# qt backend (opens separate window). Small bugs 
# import IPython

# shell = IPython.get_ipython()
# shell.enable_matplotlib(gui='qt')

# IPython widget backend
%matplotlib widget

plotter.set_file_interval(0, 9)

plt.close('all')
plotter.init_figure()
plotter.plot_counts()
plotter.plot_input(10e-3, log_offset=0.32)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

unequal:  D:\data\Lucas\20210707_lucas_test_saving\splitting_test\test_splitting_counts0008.tdms


In [51]:
# Calculate the Fourier Spectrum
plotter.fft(1e-3) 

11.759, 14.181


In [38]:
# Disable the input and log graphs
disp = False

plotter.input_ax.set_visible(disp)
plotter.log_ax.set_visible(disp)

In [42]:
# Re-enable the input and log graphs
disp = True

plotter.input_ax.set_visible(disp)
plotter.log_ax.set_visible(disp)

In [41]:
plotter.main_ax.set_visible(False)

### Find events and plot them

As of now the events are detected by the DIO value going from True to False

The timetraces near each event are loaded in, plotted and the graph is then saved as .png in a folder in the TDMS file's directory

In [45]:
from utils import make_extension, DIO_event, read_interval_from_multiple_files, AbstractCountsFile, COUNTS
        
       
event_bins = 10e-3    
    
# loop over events to find thresholding events
dio_events = []
for i in plotter.log_entries:
    if i.previous_event and i.previous_event._dio:
        if not i._dio:
            file_index = int(i.t_start // seconds_in_file)
            relative_to_file = i.t_start - file_index*seconds_in_file
            dio_events.append(DIO_event(file_index=file_index, 
                                        seconds_in_file=relative_to_file, total_seconds = i.t_start))


print('In this experiment there were %i observed encounters' % len(dio_events))

# draw counts near DIO events

left_margin = 6
right_margin = 3

event_fig = plt.figure()
for event in dio_events:
    event_data_x, event_data_y = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  event.total_seconds-left_margin, event.total_seconds+right_margin,
                                                                  10e-3)
    
    save_dir = abstractfile.dir + ('events_%s%s\\' % (abstractfile.basename, COUNTS))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    event_fig.clf()
    plt.plot(event_data_x + int((event.total_seconds-left_margin)//seconds_in_file)*seconds_in_file, event_data_y / event_bins)
    plt.scatter(event.total_seconds, 0, color='r')
    plt.savefig(save_dir + ('%i' %event.total_seconds) + '.png')
    
plt.close(event_fig)
    

In this experiment there were 11 observed encounters
D:\data\Lucas\20210707_lucas_test_saving\splitting_test\test_splitting_counts.tdms


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
from utils import read_interval_from_multiple_files
from log_entry import find_event_at_time


index = 5 # open event with this index (starting at 0)
lm = 6 # left margin
rm = 3 # right margin
fft_bin = 1e-3
binsize = 10e-3

event = dio_events[index]

fig, axes = plt.subplots(3, figsize=(8,8))

def onselect(xmin, xmax):
    axes[1].clear()
    axes[2].clear()
    zoomx, zoomy = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  xmin, xmax, binsize)
    axes[1].plot(zoomx, zoomy / binsize)

    t, s = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                  xmin, xmax, fft_bin)
    
    fft_y = np.abs(np.fft.fft(s))
    fft_x = np.arange(0, np.shape(fft_y)[0], 1) / fft_bin / np.shape(s)[0];
    
    axes[2].loglog(fft_x, fft_y)
    

datax, datay = read_interval_from_multiple_files(AbstractCountsFile(abstractfile), seconds_in_file,
                                                                    event.total_seconds-lm, event.total_seconds+rm,
                                                                    binsize)

axes[0].plot(datax + int((event.total_seconds-lm)//seconds_in_file)*seconds_in_file, datay / binsize)
event_span = SpanSelector(axes[0], onselect, 'horizontal', useblit=True,
                                rectprops=dict(alpha=0.7, facecolor='orange'), span_stays=True, minspan=0.05)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …